<a href="https://colab.research.google.com/github/gihuni99/Pytorch_Study/blob/main/Ch6_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AlexNet

## AlexNet의 구조

![image](https://github.com/gihuni99/Pytorch_Study/assets/90080065/985ccc2c-b9ef-4ad5-8692-fbc7a32deaf7)

![image](https://github.com/gihuni99/Pytorch_Study/assets/90080065/5af8c8e2-8bf8-4cfd-9946-3421dcf7aac0)

**AlexNet**은 Convolutional layer에서 Activation function으로 **ReLU함수**를 사용한다. GPU-1은 color와 상관없는 정보를 추출하기 위한 커널이 학습되고, GPU-2는 주로 color와 관련된 정보를 추출하기 위한 커널이 학습된다.



## 실습

이전의 LeNet과 Training, Test process자체는 같다. 다마 모델의 구성이 다르기 때문에 그것에 집중에서 실습을 해보자.

### 필요 라이브러리 호출

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import random
import time
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

![image](https://github.com/gihuni99/Pytorch_Study/assets/90080065/1c63b97c-74ba-4a04-98d3-fd2a7f498b6a)

AlexNet은 softmax함수를 통해 1000x1 vector의 output이 나오지만, 실습에서는 'Cat', 'Dog' 두가지 label만을 사용

### Data Pre-processing

In [ ]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase):
        return self.data_transform[phase](img)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cat_directory='/content/drive/MyDrive/Pytorch_study/data/dogs-vs-cats/Cat/'
dog_directory='/content/drive/MyDrive/Pytorch_study/data/dogs-vs-cats/Dog/'

cat_images_directory=sorted([os.path.join(cat_directory, f) for f in os.listdir(cat_directory)])
dog_images_directory=sorted([os.path.join(dog_directory, f) for f in os.listdir(dog_directory)])

images_filepaths=[*cat_images_directory,*dog_images_directory]
correct_images_filepaths=[i for i in images_filepaths if cv2.imread(i) is not None]

random.seed(42)
random.shuffle(correct_images_filepaths)
train_images_filepath=correct_images_filepaths[:400]
val_images_filepath=correct_images_filepaths[400:-10]
test_images_filepath=correct_images_filepaths[-10:]
print(len(train_images_filepath),len(val_images_filepath),len(test_images_filepath))

실습을 진행하기 전 AlexNet은 parameter가 6000만개이다. 따라서 충분한 데이터가 없으면 성능이 좋지 않은데, 실습에서 많은 데이터를 쓸 수 없기 때문에 overfitting으로 인한 성능저하가 당연히 발생할 수 밖에 없다.

- 해당 코드는 **Custom_Dataset** 클래스를 통해 image를 불러와 image preprocessing과 labeling을 진행한다. 이를 DataLoader에 전달하여 데이터를 메모리로 불러오는 것

In [ ]:
class Custom_Dataset():
    def __init__(self,file_list,transform=None,phase='train'):
        self.file_list=file_list #image가 존재하는 경로
        self.transform=transform #Data preprocessing
        self.phase=phase #'train' 또는 'val'

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path=self.file_list[idx]
        img=Image.open(img_path) #PIL을 이용하여 image불러옴
        img_transformed=self.transform(img,self.phase) #phase값에 따라 preprocessing과정이 달라짐(ImageTransform()에서 정의되어 있다.)

        label=img_path.split('/')[-1].split('.')[0] #'/content/drive/MyDrive/Pytorch_study/data/dogs-vs-cats/Cat/cat.1.jpg'의 경로에서 label추출
        if label=='dog':
            label=1
        elif label=='cat':
            label=0

        return img_transformed,label #전처리가 적용된 image와 label


In [ ]:
#preprocessing에 필요한 mean, std등의 변수 값 정의
size=256 #model input의 크기
mean=(0.485,0.456,0.406)
std=(0.229,0.224,0.225)
batch_size=32

In [ ]:
#각 dataset별로 Custom_Dataset의 객체 생성, __getitem__()함수를 통해 값을 반환하는 것이다.
train_dataset=Custom_Dataset(train_images_filepath,ImageTransform(resize=size,mean=mean,std=std),phase='train')
val_dataset=Custom_Dataset(val_images_filepath,ImageTransform(resize=size,mean=mean,std=std),phase='val')
test_dataset=Custom_Dataset(test_images_filepath,ImageTransform(resize=size,mean=mean,std=std),phase='val')

print('preprocessing된 image의 크기:', train_dataset.__getitem__(0)[0].size())
print('label:',train_dataset.__getitem__(0)[1])

### DataLoader

- 데이터를 메모리로 불러옴

In [ ]:
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
test_dataloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
dataloader_dict={'train':train_dataloader,'val':val_dataloader}

batch_iterator=iter(train_dataloader) #iter는 반복 가능한 객체에서 이터레이터를 반환
inputs,label=next(batch_iterator) #iterator에서 값을 차례로 반환
print(inputs.size())
print(label)

### Model Architecture

![image](https://github.com/gihuni99/Pytorch_Study/assets/90080065/2af86c58-9f16-4889-8e7a-1c355c5b3f3b)

- ReLU(inplace=True): inplace는 결과값을 새로운 변수에 저장하는 것이 아닌, 기존 데이터를 대체한다는 의미

- self.avgpool = nn.AdaptiveAvgPool2d((6, 6)): AvgPool2d는 nn.AvgPool2d(2,stride=3) 또는 nn.AvgPool2d((2,1),stride=(3,2))로 사용하여 kernel size와 stride를 지정해준다.((2,1)은 2x1 kernel, stride=(3,2)는 H방향으로 stride=3, W방향으로 stride=2를 의미한다. 반면 nn.AdaptiveAvgPool2d는  pooling이 끝날 때의 출력을 정의한다. 즉, nn.AdaptiveAvgPool2d((6, 6))의 결과는 6x6이 된다.

In [ ]:
class AlexNet(nn.Module):
    def __init__(self) -> None: #->None은 함수가 반환하는 데이터 타입을 나타내는 주석과 같다. None은 반환하지 않음을 의미
        super(AlexNet, self).__init__() #작성하지 않아도 nn.Module을 상속받는다(가독성을 높이기 위한 코드)
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256*6*6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 2),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor: #Tensor를 반환함
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

### Model Object 생성

In [ ]:
model=AlexNet()
model.to(device)

### Optimizer & Loss Function

In [ ]:
optimizer=optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
criterion=nn.CrossEntropyLoss().to(device)

### Model Architecture 확인

In [ ]:
from torchsummary import summary
summary(model,input_size=(3,256,256))

### Training

In [ ]:
def train_model(model,dataloader_dict,criterion,optimizer,num_epoch):

    since=time.time()
    best_acc=0.0

    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch+1,num_epoch))
        print('-'*20)

        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            epoch_loss=0.0
            epoch_corrects=0

            for inputs,labels in tqdm(dataloader_dict[phase]):
                inputs=inputs.to(device)
                labels=labels.to(device)
                optimizer.zero_grad() #back propagation 전에 항상 gradient를 0으로 초기화해주어야 한다.

                with torch.set_grad_enabled(phase=='train'): #autograd 활성화
                    outputs=model(inputs)
                    _, preds=torch.max(outputs,1) # preds=torch.max(outputs,1)로 쓰면 preds[0]은 최대값을, preds[1]은 최대값의 인덱스 즉 label을 나타낸다.
                    loss=criterion(outputs,labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item()*inputs.size(0) #inputs.size(0)은 input의 행, 즉 batch size를 의미한다. loss는 batch전체의 평균 loss이므로 batch size만큼 곱해준다.
                    epoch_corrects+=torch.sum(preds==labels.data) #예측과 정답이 얼마나 정확한지 측정

            epoch_loss=epoch_loss/len(dataloader_dict[phase].dataset) #epoch의 평균 loss를 구한다.
            epoch_acc=epoch_corrects.double()/len(dataloader_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
    time_elapsed=time.time()-since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed//60,time_elapsed%60))
    return model

In [ ]:
num_epoch=10
model=train_model(model,dataloader_dict,criterion,optimizer,num_epoch)

### Evaluation

In [ ]:
import pandas as pd
id_list = []
pred_list = []
_id = 0
with torch.no_grad():
    for test_path in tqdm(test_images_filepath): #test dataset사용
        img = Image.open(test_path)
        _id = test_path.split('/')[-1].split('.')[1]# data의 index가져오기 (Ex dog.113.jpg라는 이미지 이름에서 113)
        transform = ImageTransform(size, mean, std)
        img = transform(img, phase='val')# test dataset pre processing
        img = img.unsqueeze(0)
        img = img.to(device)

        model.eval()
        outputs = model(img)
        preds = F.softmax(outputs, dim=1)[:, 1].tolist()

        id_list.append(_id)
        pred_list.append(preds[0])

res = pd.DataFrame({
    'id': id_list,
    'label': pred_list
}) # dataframe에 이미지의 id(번호)와 레이블 저장
res.to_csv('/content/drive/MyDrive/Pytorch_study/alexnet.csv', index=False) #이미지의 id와 레이블을 alexnet.csv 파일에 저장

### Dataframe 확인

In [ ]:
res.head(10)

### Evaluation 결과 시각화

In [ ]:
class_ = classes = {0:'cat', 1:'dog'}
def display_image_grid(images_filepaths, predicted_labels=(), cols=5):
    rows = len(images_filepaths) // cols
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
    for i, image_filepath in enumerate(images_filepaths):
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        a = random.choice(res['id'].values)
        label = res.loc[res['id'] == a, 'label'].values[0]
        if label > 0.5:
            label = 1
        else:
            label = 0
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_title(class_[label])
        ax.ravel()[i].set_axis_off()
    plt.tight_layout()
    plt.show()

- parameter가 많지만, 실습을 위해 데이터셋 양이 적기 때문에 성능이 좋지 않다.

In [ ]:
display_image_grid(test_images_filepath)